### Запуск QtLab

In [1]:
%run D:\qtlab_replacement\init.py
%matplotlib qt5
import setup_logging
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
from save_pkl import *
from matplotlib import interactive
interactive(True)
import os
import matplotlib.patches as plm

Failed loading module Signal_Hound_SA:  [WinError 126] The specified module could not be found


# Спектроскопия

In [2]:
pna = Agilent_N5242A('pna', address = 'TCPIP0::10.20.61.48::inst0::INSTR')
lo1 = Agilent_E8257D('lo1', address = 'TCPIP0::10.20.61.59::inst0::INSTR')

In [3]:
#current_src1 = Yokogawa_GS210(address = 'GPIB0::1::INSTR')
#current_src2 = Yokogawa_GS210(address = 'GPIB0::2::INSTR')
current_src = Yokogawa_GS210(address = 'GPIB0::2::INSTR')

In [4]:
dac = nndac('TCPIP0::10.20.61.12::1000::SOCKET')

TypeError: __init__() takes 1 positional argument but 2 were given

In [119]:
# векторный анализатор в режим линейного свипа
pna.set_sweep_mode("LIN")


True

In [4]:
#if not ('pna' in locals()): pna = Agilent_N5242A('pna', address = 'PNA')
#if not ('lo1' in locals()): lo1 = Agilent_E8257D('lo1', address = 'PSG1')
sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')
#sa = Signal_Hound_SA(name='SA124', serial=61660066)
#lo_ex = Labbrick(name='lo_ex', serial=15249)
lo_ex = lo1
lo_ro = pna
#lo_ro = Labbrick(name='lo_ro', serial=15257)
awg_tek= Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')

#CHASSIS = 1
#SLOT_IN = 16
#SLOT_OUT = 7
#awg = Keysight_M3202A('awg', CHASSIS, SLOT_OUT)
#adc = Keysight_M3102A('adc', CHASSIS, SLOT_IN)
#awg_adc = 


root: WARNING  Tektronix_AWG5014 : get all not yet fully functional


## Резонаторная спектроскопия

### Одномерный скан при фиксированном токе 

In [136]:
lo1.set_status(0)

True

In [138]:
pna.set_nop(501)
pna.set_xlim(6.4e9,6.5e9)
pna.set_power(-40)
pna.set_bandwidth(100)
s21=pna.get_tracedata()
fr=pna.get_freqpoints()
plt.figure(1)
plt.plot(fr[:len(fr)],s21[0])
plt.figure(2)
plt.plot(fr[:len(fr)-1],np.diff( np.unwrap(s21[1])))

In [27]:
qubit_id

1

In [5]:
qubit_params = {4: {'Fr': 6.890e9,
                    'F01_max': 7.45e9,
                    'Iw_q1c':-0.5e-3},#1.699e-3},
                    #'Iw_q2c':-0.26e-3},
                1: {'Fr': 6.4225e9,  
                    'F01_max': 7.9509e9, 
                    'Iw_q1c':0.0e-3},
                2: {'Fr': 6.4225e9,  
                    'F01_max': 7.6975e9, 
                    'Iw_q1c':0.0e-3},
                11: {'Fr': 6.423e9,  
                    'F01_max': 7.8312e9, 
                    'Iw_q1c':0.0e-3},}
qubit_id=1

In [9]:
lo_ex.set_status(1)

True

In [11]:
# когда спектроскопия не рабтает и всё плохо, тыкай сюды!
# вектнй анлизатр цепей - в режим свипа по частоте
pna.set_sweep_mode("LIN")
# смесители - открыть
#awg_tek = awg
#for channel in range(0,4):
for channel in range(1,3):
    awg_tek.set_amplitude(0.15, channel=channel)
    awg_tek.set_offset(0.15, channel=channel)
    awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=channel)
    awg_tek.set_output(1, channel=channel)

for channel in range(3,5):
    awg_tek.set_amplitude(0.3, channel=channel)
    awg_tek.set_offset(0.3, channel=channel)
    awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=channel)
    awg_tek.set_output(1, channel=channel)
#     
# Задаём постояннmtые смещения на каналы 1 и 3
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=1)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=2)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=3)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=4)

In [43]:
pna.set_xlim(6e9, 8e9)
pna.set_nop(2001)
pna.set_power

<function instrument.Instrument.add_parameter.<locals>.<lambda>>

In [44]:
resolution_bw=10
tone1_power = -40

In [7]:
def curr(current):
    lo1.set_status(0)
    pna.set_average(0)
    old_power = pna.get_power()
    pna.set_power(-25)
    pna.set_nop(101)
    dfr=0.05e9
    
    pna.set_xlim(qubit_params[qubit_id]['Fr']-dfr,  qubit_params[qubit_id]['Fr']+dfr)
    pna.set_bandwidth(200)
    #current_src.set_current(current)
    S21   = pna.get_tracedata()
    freqs = pna.get_freqpoints()
    pna.set_bandwidth(resolution_bw)
    pna.set_power(old_power)
    pna.set_nop(1)
    S21g = np.abs(np.gradient(S21[0]*np.exp(1j*S21[1])))**2
    pna.set_xlim(freqs[np.argmin(S21g)]-0.2e6,freqs[np.argmin(S21g)]-0.2e6)
    lo1.set_status(1)
    #print (freqs[np.argmin(S21g)])
    #plt.plot(S21g)

In [12]:
curr(0)

In [162]:
lo1.set_status(0)
current_src.set_status(1)
pna.set_power(-25)
pna.set_nop(701)
pna.set_xlim(6.3e9, 7.0e9)
pna.set_bandwidth(200)
currents = np.linspace(-0.8e-3, 0.8e-3, 201)
sweep.sweep(pna, (currents, current_src.set_current, 'Coil current', 'A'), filename='Xmon 2Q anti')

Started at:  Jul 02 2018 18:14:00
Time left: 0 h 0 m 0.0 s,	parameter values: Coil current: 0.0008,	average cycle time:  3.6s	, plot_update_time: 0.35s5sss5s
Elapsed time: 0 h 12 m 3.35 s


{'S-parameter': (('Coil current', 'Frequency'),
  (array([-8.00000000e-04, -7.92000000e-04, -7.84000000e-04, -7.76000000e-04,
          -7.68000000e-04, -7.60000000e-04, -7.52000000e-04, -7.44000000e-04,
          -7.36000000e-04, -7.28000000e-04, -7.20000000e-04, -7.12000000e-04,
          -7.04000000e-04, -6.96000000e-04, -6.88000000e-04, -6.80000000e-04,
          -6.72000000e-04, -6.64000000e-04, -6.56000000e-04, -6.48000000e-04,
          -6.40000000e-04, -6.32000000e-04, -6.24000000e-04, -6.16000000e-04,
          -6.08000000e-04, -6.00000000e-04, -5.92000000e-04, -5.84000000e-04,
          -5.76000000e-04, -5.68000000e-04, -5.60000000e-04, -5.52000000e-04,
          -5.44000000e-04, -5.36000000e-04, -5.28000000e-04, -5.20000000e-04,
          -5.12000000e-04, -5.04000000e-04, -4.96000000e-04, -4.88000000e-04,
          -4.80000000e-04, -4.72000000e-04, -4.64000000e-04, -4.56000000e-04,
          -4.48000000e-04, -4.40000000e-04, -4.32000000e-04, -4.24000000e-04,
          -4.160

# two-tone 1D

In [14]:
qubit_params[qubit_id]

{'F01_max': 7450000000.0, 'Fr': 6873130000.0, 'Iw_q1c': -0.0005}

In [175]:
import imp
imp.reload(sweep)
imp.reload(plotting)

<module 'plotting' from 'D:\\qtlab_replacement\\scripts\\plotting.py'>

In [16]:
qubit_id=1

ro_power = -45
ex_power = -15
span =500e6
step  =1e6

#current_src1.set_status(1)
#current_src1.set_current(qubit_params[qubit_id]['Iw_q1c'])
#curr(qubit_params[qubit_id]['Iw_q1c'])
resolution_bw = 10
#curr(0.0)

lo1.set_status(1)
lo1.set_power(ex_power)
pna.set_bandwidth(10)

pna.set_power(ro_power)

pna.set_nop(1)
pna.set_centerfreq(qubit_params[qubit_id]['Fr'])

#current_src1.set_current(qubit_params[qubit_id]['Iw_q1c'])
#set_curr_start(qubit_params[qubit_id]['Iw_q1c'],current_src1)

#ex_freqs = qubit_params[qubit_id]['F01_max']+np.arange(-span/2, span/2, step)
ex_freqs = np.linspace(7.5e9, 8e9, 501)

meas=sweep.sweep(pna, (ex_freqs, lo1.set_frequency, '2nd tone frequency'), 
                 filename='TwoTone1D_qubit{:d}_ro{:.2f}_ex{:.2f}'.format(qubit_id, ro_power, ex_power))
lo1.set_status(0)

A Jupyter Widget

Started at:  Jul 05 2018 14:45:04


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2169: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2170: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2172: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)



Elapsed time: 0 h 1 m 38.87 s


True

In [13]:
qubit_id = 2
pna.set_power(-45)
lo1.set_status(1)
lo1.set_power(-15)
pna.set_bandwidth(7)
#freq = qubit_params[qubit_id]['F01_max']+np.arange(-400e6, 100e6, 5e6)
freq = np.linspace(7.0e9, 8e9, 201)

#current_max=qubit_params[qubit_id]['Iw_q1c']
currents = np.arange(-0.8e-3, +0.8e-3, 0.04e-3)
current_src.set_status(1)
#set_curr_start(currents[0],current_src1)
#currents=np.linspace(-1.850e-3,-1.825e-3,current_nop)
#set_curr_start(currents[0],current_src1)

#for i in range(3,len(res_fr)):
# dfr =2
#    tone1_freq_min, tone1_freq_max = qubit_params[qubit_id]['Fr']-dfr,  qubit_params[qubit_id]['Fr']+dfr
pna.set_nop(1)
pna.set_centerfreq( qubit_params[qubit_id]['Fr'])
pna.set_nop(1)
#currents=np.linspace(qubit_params[qubit_id]['Iw_q1c']-25e-5,qubit_params[qubit_id]['Iw_q1c']+25e-5,51)
sweep.sweep(pna, #(currents, 
                 # current_src.set_current,'current_qubit{}'.format(qubit_id)), 
            (currents, curr, 'current_qubit{}'.format(qubit_id), 'A'),
            (freq, lo1.set_frequency, '2nd tone frequency', 'Hz'), 
            filename='qubit {3} tone_fr{0}_t1p{1}_t2p{2}'.format("%.2f" % (qubit_params[1]['Fr']/1e9),pna.get_power(),lo1.get_power(), 
                                                                 qubit_id))
           # bot=(True,151))
    
#set_curr_finish(current_src1)
#set_curr_finish(current_src2)
#current_src1.set_status(0)
#current_src2.set_status(0)

A Jupyter Widget

Started at:  Jul 05 2018 14:43:49


Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "D:\qtlab_replacement\scripts\sweep.py", line 202, in main_sweep_loop
    ascii_file.write(data_to_str(fmt(mpoint_m_flat))+'\n')
ValueError: I/O operation on closed file.



{'S-parameter': (('current_qubit2', '2nd tone frequency'),
  (array([ -8.00000000e-04,  -7.60000000e-04,  -7.20000000e-04,
           -6.80000000e-04,  -6.40000000e-04,  -6.00000000e-04,
           -5.60000000e-04,  -5.20000000e-04,  -4.80000000e-04,
           -4.40000000e-04,  -4.00000000e-04,  -3.60000000e-04,
           -3.20000000e-04,  -2.80000000e-04,  -2.40000000e-04,
           -2.00000000e-04,  -1.60000000e-04,  -1.20000000e-04,
           -8.00000000e-05,  -4.00000000e-05,  -1.08420217e-19,
            4.00000000e-05,   8.00000000e-05,   1.20000000e-04,
            1.60000000e-04,   2.00000000e-04,   2.40000000e-04,
            2.80000000e-04,   3.20000000e-04,   3.60000000e-04,
            4.00000000e-04,   4.40000000e-04,   4.80000000e-04,
            5.20000000e-04,   5.60000000e-04,   6.00000000e-04,
            6.40000000e-04,   6.80000000e-04,   7.20000000e-04,
            7.60000000e-04]),
   array([  7.00000000e+09,   7.00500000e+09,   7.01000000e+09,
            7.0

In [132]:
fig = plt.figure()

In [71]:
curr(0)

In [ ]:
#pna.set_power(-55)
current_src1.set_status(1)
current_src2.set_status(1)
set_curr_start(-0.26e-3,current_src2)
#current_src2.set_current(-0.26e-3)
lo1.set_status(1)
lo1.set_power(-20)
#current_src2.set_status(1)
freq = qubit_params[qubit_id]['F01_min']+np.linspace(-250e6, 100e6, 351)
#freq = np.linspace(4.71e9, 4.76e9, 401)
current_nop=201
current_max=-0.1e-3
currents=np.linspace(current_max-0.13e-3,current_max+0.13e-3,current_nop)
set_curr_start(currents[0],current_src1)
#for i in range(3,len(res_fr)):
# dfr =2
#    tone1_freq_min, tone1_freq_max = qubit_params[qubit_id]['Fr']-dfr,  qubit_params[qubit_id]['Fr']+dfr
pna.set_nop(1)
pna.set_xlim( qubit_params[qubit_id]['Fr'], qubit_params[qubit_id]['Fr'])
#currents=np.linspace(qubit_params[qubit_id]['Iw_q1c']-25e-5,qubit_params[qubit_id]['Iw_q1c']+25e-5,51)
sweep.sweep(pna, (currents, 
                  current_src1.set_current,'current_qubit{}'.format(qubit_id)), 
            (freq, lo1.set_frequency, '2nd tone frequency'), 
            filename='qubit {3} tone_fr{0}_t1p{1}_t2p{2}'.format("%.2f" % (qubit_params[1]['Fr']/1e9),pna.get_power(),lo1.get_power(), qubit_id),
            bot=(True,351))
    
set_curr_finish(current_src1)
set_curr_finish(current_src2)
current_src1.set_status(0)
current_src2.set_status(0)

In [ ]:
qubit_params[qubit_id]['Iw_q2c']

In [ ]:
-0.00173,4.74e9

Если сорвалась двухтонка ЖНИ

In [ ]:
#Если сорвалась двухтонка ЖНИ
set_curr_finish(current_src1)
#set_curr_finish(current_src2)
current_src1.set_status(0)
#current_src2.set_status(0)

In [ ]:
for qubit_id in [1]:
    pna.set_power(-50)
    lo1.set_status(1)
    lo1.set_power(-20)
    current_src1.set_status(1)
    #freq = qubit_params[qubit_id]['F01_min']+np.linspace(-250e6, 100e6, 351)
    freq = np.linspace(4.71e9, 4.76e9, 401)
    currents=np.linspace(0.185e-3,0.26e-3,91)
    #for i in range(3,len(res_fr)):
   # dfr =2
#     tone1_freq_min, tone1_freq_max = qubit_params[qubit_id]['Fr']-dfr,  qubit_params[qubit_id]['Fr']+dfr
    pna.set_xlim( qubit_params[qubit_id]['Fr'], qubit_params[qubit_id]['Fr'])
    #currents=np.linspace(qubit_params[qubit_id]['Iw_q1c']-25e-5,qubit_params[qubit_id]['Iw_q1c']+25e-5,51)
    sweep.sweep(pna, (currents, 
                      current_src1.set_current,'current'), 
                (freq, lo1.set_frequency, '2nd tone frequency'), 
                filename='qubit {3} tone_fr{0}_t1p{1}_t2p{2}'.format("%.2f" % (qubit_params[1]['Fr']/1e9),pna.get_power(),lo1.get_power(), qubit_id),
               bot=(True,401))

In [ ]:
import pickle as pkl
with open(r'C:\Users\demouser\Desktop\Quantum\Data\2018-05-30\17-14-31-2nd tone_fr6.53_t1p-25.0_t2p-5.0\S-parameter 2nd tone_fr6.53_t1p-25.0_t2p-5.0.pkl', 'rb') as f:
    data_new = pickle.load(f)
data_new[0]

In [ ]:

with open(r'D:\qtlab\ReiData\data\2018-06-21\00-02-52-qubit 1 tone_fr6.53_t1p-55.0_t2p-15.0\S-parameter qubit 1 tone_fr6.53_t1p-55.0_t2p-15.0.pkl', 'rb') as f:
    data_new = pickle.load(f)     
plt.pcolormesh(data_new[1]['S-parameter'][1][0],data_new[1]['S-parameter'][1][1], np.angle(data_new[1]['S-parameter'][2].T), cmap='RdBu')
plt.show()

In [ ]:
with open(r'C:\Users\demouser\Desktop\Quantum\Data\2018-05-30\13-28-36-2nd tone_fr6.53_t1p-25.0_t2p-5.0\S-parameter 2nd tone_fr6.53_t1p-25.0_t2p-5.0.pkl', 'rb') as f1:
    data_new = pickle.load(f1)     
plt.pcolormesh(data_new[1]['S-parameter'][1][0],data_new[1]['S-parameter'][1][1], np.abs(data_new[1]['S-parameter'][2].T), cmap='RdBu')
plt.show()

In [ ]:
current_src1.set_current(qubit_params[qubit_id]['Iw'])

In [20]:
adc.stop()

# Импульсы

In [6]:
#if not ('pna' in locals()): pna = Agilent_N5242A('pna', address = 'PNA')
#if not ('lo1' in locals()): lo1 = Agilent_E8257D('lo1', address = 'PSG1')
sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')
#sa = Signal_Hound_SA(name='SA124', serial=61660066)
#lo_ex = Labbrick(name='lo_ex', serial=15249)
lo_ex = lo1
lo_ro = pna
#lo_ro = Labbrick(name='lo_ro', serial=15257)
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
adc = Spectrum_M3i2132('adc')

root: WARNING  Tektronix_AWG5014 : get all not yet fully functional


### Создаём девайсы

### Загружаем импульсные скрипты всякие

In [7]:
import awg_digital
import awg_iq
import data_reduce
import imp
import fitting
import plotting
import save_pkl
import tomography as tomography_new
import tomography_legacy as tomography
import sweep
import awg_iq_multi
imp.reload(awg_digital)
imp.reload(awg_iq)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(save_pkl)
imp.reload(tomography)
imp.reload(tomography_new)
imp.reload(sweep)
imp.reload(awg_iq_multi)

<module 'awg_iq_multi' from 'D:\\qtlab_replacement\\scripts\\awg_iq_multi.py'>

### Осцилляторы

In [8]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм
#lo_ex_pow = 14
#lo_ex_pow = 14
lo_ex_pow = 14
#lo_ex = lo1
lo_ex.set_status(1)
#lo_ex.set_power(lo_ex_pow)
#pna как lo2 (excitation)
#Port1 - выход
#Мощность гетеродина для считывания 10-13 дБм, с учётом разветвителя 13-16 dBm
#lo_ro_pow = 16
lo_ro_pow = 16
lo_ro.set_power(lo_ro_pow)

#lo_ex = pna
pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
lo_ex.set_power(lo_ex_pow)

True

### AWGшки

In [9]:
# промежуточные частоты для гетеродинной схемы:

lo_freq = 7.77e9
lo1.set_frequency(lo_freq)
#ex_if = 150e6
ex_if = lo_freq-qubit_params[qubit_id]['F01_max']
#ex_sideband_id = -1 if ex_if>0 else +1
ex_sideband_id = -1
ex_if = np.abs(ex_if)
ro_if = 75e6
print (ex_if)
# клоки генератора и оцифровщика
ex_clock = 1e9
ro_clock = 0.5e9

rep_rate = 20e3 # частота повторений эксперимента

awg_tek.check_cached = True
awg_tek.stop()
awg_tek.set_clock(ex_clock) # клок всех авгшк
awg_tek.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
#awg_tek.run()
# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.

# миксеры подключены к AWG5014 через 6 дБ аттенюаторы. Ставим амплитуду 2В.
for channel in range(1,3):
    awg_tek.set_amplitude(0.1, channel=channel)
    awg_tek.set_offset(0, channel=channel)
    awg_tek.set_output(1, channel=channel)
for channel in range(3,5):
    awg_tek.set_amplitude(0.2, channel=channel)
    awg_tek.set_offset(0, channel=channel)
    awg_tek.set_output(1, channel=channel)
# Выходы 1, 2 тектроникс: возбуждение (кубитный тон) через 6 дБ аттенюатор
# Выходы 3, 4 тектроникс: считывание (резонаторный тон) через 6 дБ аттенюатор
iq_ex = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 2, 1, lo_ex)
#iq_ex = awg_iq.awg_iq(awg_tek, awg_tek, 2, 1, lo_ex) calibrated
iq_ro = awg_iq.awg_iq(awg_tek, awg_tek, 3, 4, lo_ro)
iq_ex.carriers['00-1-01'] = awg_iq_multi.carrier(iq_ex)
iq_ex.carriers['00-1-10'] = awg_iq_multi.carrier(iq_ex)
iq_ex.carriers['00-2-11'] = awg_iq_multi.carrier(iq_ex)
iq_ex.carriers['00-1-01'].set_frequency(qubit_params[1]['F01_max'])
iq_ex.carriers['00-1-10'].set_frequency(qubit_params[2]['F01_max'])
iq_ex.carriers['00-2-11'].set_frequency(qubit_params[11]['F01_max'])
iq_ro.set_if(ro_if)
#iq_ex.set_sideband_id(ex_sideband_id)
iq_ro.set_sideband_id(-1)
#iq_ex.set_frequency(qubit_params[qubit_id]['F01_max'])
#iq_ex.set_lo_frequency(lo_freq)
iq_ro.set_frequency(qubit_params[qubit_id]['Fr'])

# К первому маркере первого канала тектроникса подсоединён триггер оцифровщика.
# Надо подавать какой-нибудь триггер, что-ли.
# А ещё нужно подавать клок. К сожалению наш оцифровщик не может в 1ГГц клок, ему нужен 500МГц клок.
# Будем подавать 500 МГц с цифрового выхода тектроникса.
awg_tek.set_marker1_low(-0.1, channel=1)
awg_tek.set_marker2_low(-0.1, channel=1)
awg_tek.set_marker1_high(1.2, channel=1)
awg_tek.set_marker2_high(1.2, channel=1)
awg_tek.set_marker1_low(-0.1, channel=2)
awg_tek.set_marker2_low(-0.1, channel=2) 
awg_tek.set_marker1_high(0.1, channel=2) # с клока хватит и такой амплитуды
awg_tek.set_marker2_high(0.1, channel=2) # с клока хватит и такой амплитуды
ro_trg = awg_digital.awg_digital(awg_tek, 1)
osc_trg = awg_digital.awg_digital(awg_tek, 5)
ro_adc_clock = awg_digital.awg_digital(awg_tek, 2) #
osc_adc_clock = awg_digital.awg_digital(awg_tek, 6) #

# тупо клок
ro_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
osc_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
awg_channels = {'iq_ex_00-1-01':iq_ex.carriers['00-1-01'], 
                'iq_ex_00-1-10':iq_ex.carriers['00-1-10'], 
                'iq_ex_00-2-11':iq_ex.carriers['00-2-11'], 
                'iq_ro':iq_ro, 
                'ro_trg':ro_trg, 
                'osc_trg':osc_trg}
trg_length = 4e-9
pg = tomography_new.pulses(awg_channels)
awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
awg_tek.run()


root: WARNING  Tektronix_AWG5014 : Clock set to 1000000000.0. This is not fully functional yet. To avoid problems, it is better not to change the clock during operation


180900000.0


### Оцифровщик

In [10]:
# настройки оцифровщика
adc.set_clock(ro_clock)
adc.reset()            
adc.set_timeout(10000)


#adc.set_spc_samplerate(smp_rate)
#External trigger
adc.set_trigger_ext0_level0(400) # logical 0 value to 200 mV
adc.set_trigger_ext0_level1(600) # logical 1 value to 600 mV
adc.trigger_termination_50Ohm()
adc.trigger_mode_pos()
adc.set_trigger_ext0_pulsewidth(0)
adc.disable_trigger_output()
adc.select_channel01()
adc.set_multi_mode()
#adc.set_segmentsize()
adc.set_trigger_ORmask_tmask_ext0()
adc.set_trigger_ANDmask_tmask_ext0()
adc.set_trigger_ORmask_tmask_NO_ch0()
adc.set_trigger_ORmask_tmask_NO_ch1()
adc.set_trigger_ANDmask_tmask_NO_ch0()
adc.set_trigger_ANDmask_tmask_NO_ch1()
# усилитель на оцифровщике
adc.set_input_amp_ch0(50)
adc.set_input_amp_ch1(50)
adc.set_input_offset_ch0(0)
adc.set_input_offset_ch1(0)

#External clock
adc.set_reference_clock(adc.get_clock())

# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
adc_reducer = data_reduce.data_reduce(adc)
adc_reducer.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)
adc_reducer.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.get_if())
adc_reducer.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.get_if())


### Калибровка миксеров

In [11]:
#sa.set_ref(0)
iq_ex.do_calibration(sa)
iq_ro.get_calibration(sa)

d:\qtlab\ReiData\data/calibrations//IQ-dc-lo7.77GHz.pkl
d:\qtlab\ReiData\data/calibrations//IQ-if1.8e+08-rf8e+09-sb-0.pkl
d:\qtlab\ReiData\data/calibrations//IQ-if-7.2e+07-rf7.7e+09-sb-0.pkl
d:\qtlab\ReiData\data/calibrations//IQ-if6.1e+07-rf7.8e+09-sb-0.pkl
[Errno 2] No such file or directory: 'd:\\qtlab\\ReiData\\data/calibrations//IQ-if6.1e+07-rf7.8e+09-sb-0.pkl'
dc: -1.51e-01+1.21e-01j	I:  5.00e-01	Q:-5.00e-01+5.00e-02j	B: -38.15 G: -12.04, C:0.00
[-38.15332413 -51.12865067 -12.04425907]
dc: -1.51e-01+1.21e-01j	I:  5.00e-01	Q:-5.25e-01+5.00e-02j	B: -39.14 G: -11.84, C:0.00
[-39.13913727 -54.48180771 -11.83590126]
dc: -1.51e-01+1.21e-01j	I:  5.00e-01	Q:-5.00e-01+5.25e-02j	B: -37.72 G: -12.04, C:0.00
[-37.7226181  -52.2994957  -12.03881645]
dc: -1.51e-01+1.21e-01j	I:  5.00e-01	Q:-5.25e-01+4.75e-02j	B: -39.68 G: -11.83, C:0.00
[-39.67713547 -50.9389534  -11.83279133]
dc: -1.51e-01+1.21e-01j	I:  5.00e-01	Q:-5.37e-01+4.50e-02j	B: -39.58 G: -11.72, C:0.00
[-39.57762146 -51.51193619 -11.7

dc: -1.51e-01+1.21e-01j	I:  5.00e-01	Q:-5.23e-01+5.82e-03j	B: -70.65 G: -11.85, C:0.00
[-70.65184021 -50.20037079 -11.85067272]
dc: -1.51e-01+1.21e-01j	I:  5.00e-01	Q:-5.23e-01+5.86e-03j	B: -70.50 G: -11.85, C:0.00
[-70.50344849 -48.47909164 -11.8545599 ]
dc: -1.51e-01+1.21e-01j	I:  5.00e-01	Q:-5.23e-01+5.85e-03j	B: -69.91 G: -11.86, C:0.00
[-69.9123764  -48.89969254 -11.85611439]
dc: -1.51e-01+1.21e-01j	I:  5.00e-01	Q:-5.23e-01+5.85e-03j	B: -70.51 G: -11.85, C:0.00
[-70.51324463 -50.75080872 -11.85067272]
dc: -1.51e-01+1.21e-01j	I:  5.00e-01	Q:-5.23e-01+5.83e-03j	B: -71.09 G: -11.85, C:0.00
[-71.09365845 -51.37355042 -11.85144997]
dc: -1.51e-01+1.21e-01j	I:  5.00e-01	Q:-5.23e-01+5.82e-03j	B: -70.72 G: -11.85, C:0.00
[-70.71929169 -49.11349106 -11.85067272]
dc: -1.51e-01+1.21e-01j	I:  5.00e-01	Q:-5.23e-01+5.82e-03j	B: -71.26 G: -11.85, C:0.00
[-71.2614975  -49.55819702 -11.85300541]
dc: -1.51e-01+1.21e-01j	I:  5.00e-01	Q:-5.23e-01+5.83e-03j	B: -46.06 G: -11.77, C:0.00
[-73.32687378 -71

{'I': (0.40890778708739273+0.58168590975861856j),
 'Q': (0.57704514151321362+0.46965931450731768j),
 'dc': (0.14287743083922338+0.082917249355867459j),
 'num_sidebands': 7,
 'score': -45.528546445887258}

In [21]:
sa.set_xlim(6.4e9, 6.5e9)
sa.set_nop(5001)
sa.set_bandwidth(1e5)
sa.set_bandwidth_video(1e4)
P = sa.measure()['Power']
f = sa.get_points()['Power'][0][1]
plt.plot(f, P)

### Длительность и амплитуда считывающих импульсов

In [12]:
def set_adc_nop(ro_adc_length):
    adc.stop()
    signal_size = int(np.ceil(4./3.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    posttrigger = nop*15/16
    
    print (nop)
    #adc.set_trigger_delay(25000-nop)
    adc.set_nop(nop)
    adc.set_post_trigger(posttrigger)

In [13]:
ro_adc_length =0.4e-6        # Вот такой длины считываем (опухоль Фёдорова!!)
ro_amplitude =0.2
#ro_adc_length = 0.3e-6
#ro_amplitude = 0.6
#ro_amplitude = 1.0
ro_dac_length = 0.6e-6
#ro_dac_length = 0.4e-6
adc.set_nums(10000)
#adc.set_software_nums_multi(1)
set_adc_nop(ro_adc_length)


512


In [14]:
#adc.set_nop(25000)
adc.set_nums(100000)
#plt.plot(np.mean(adc.measure()['Voltage'], axis=0))

True

In [16]:
pg.set_seq([pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', 0.2e-6, pg.rect, 1.0)])


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:654: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


In [138]:
pg.set_seq([ pg.p('iq_ex', 1e-6, pg.rect, 0.9)])

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:654: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


In [24]:
imp.reload(plotting)

<module 'plotting' from 'D:\\qtlab_replacement\\scripts\\plotting.py'>

### Прохождение считывающих испульсов разной амплитуды

In [26]:
adc.stop()
def set_pulse_amplitude(x):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    pg.set_seq([pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, x)])
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, 
                          (np.linspace(0, 1.,20), set_pulse_amplitude, 'Readout amplitude'), 
                          filename='Readout pulse passthrough',
                          plot_separate_thread=True)

A Jupyter Widget

Started at:  Jul 05 2018 15:18:11


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2169: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2170: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2172: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)
D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:655: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 1 m 39.48 s


In [ ]:
pulse_fft = np.fft.fftshift(np.fft.fft(np.fft.fftshift(np.mean(adc.measure()['Voltage'], axis=0))))
plt.plot(np.abs(pulse_fft)**2)

In [ ]:
lo1.set_status(1)

In [ ]:
current_src1.set_status(1)
set_curr_start(qubit_params[qubit_id]['Iw_q1c'],current_src1)

In [124]:
set_curr_finish(current_src1)
current_src1.set_status(0)

NameError: name 'set_curr_finish' is not defined

In [ ]:
current_src1.set_current(qubit_params[qubit_id]['Iw'])

In [123]:
current_src1.get_current()

-0.00215

### Осцилляции Раби

In [15]:
rabi_channels = ['iq_ex_00-1-01', 'iq_ex_00-1-10', 'iq_ex_00-2-11']
for rabi_channel in rabi_channels:
    measurement_name = 'Rabi channel {}'.format(rabi_channel)

    ex_amplitude =0.4
    pause_length = 0
    lengths = np.arange(0., 0.2e-6, 0.002e-6)
    readout_begin = np.max(lengths)
    def set_ex_length(length):
        awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
        sequence = [pg.p(None, readout_begin-length), 
                    pg.p(rabi_channel, length, pg.rect, ex_amplitude), 
                    pg.p(None, pause_length), 
                    pg.p('ro_trg', trg_length, pg.rect, 1), 
                    pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
        pg.set_seq(sequence)
        awg_tek.run()

    measurement = sweep.sweep(adc_reducer, (lengths, set_ex_length, 'Rabi pulse length', 's'), filename=measurement_name)


    measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)


    annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                                 fitted_parameters['freq'], 
                                                                                 fitted_parameters['decay'])
    save_pkl.save_pkl({'type':'Rabi','name': 'qubit' }, measurement_fitted, annotation=annotation, filename=measurement_name)

A Jupyter Widget

Started at:  Jul 06 2018 11:38:18
6


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2169: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2170: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2172: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)
D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:655: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 15 m 17.53 s
6


A Jupyter Widget

...


Elapsed time: 0 h 17 m 30.81 s
6


A Jupyter Widget

Started at:  Jul 06 2018 12:11:19
6


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2169: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2170: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2172: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)
D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:655: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 17 m 26.08 s
6


In [17]:
#Rabi 2D
rabi_channels = ['iq_ex_00-1-01', 'iq_ex_00-1-10', 'iq_ex_00-2-11']
for rabi_channel in rabi_channels[-1:]:
    measurement_name = 'Rabi channel {}'.format(rabi_channel)

    #ex_amplitude =0.4
    pause_length = 0
    ex_amps = np.arange(0., 0.8, 0.02)
    lengths = np.arange(0., 0.1e-6, 0.004e-6)
    readout_begin = np.max(lengths)
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    class pulse_setter:
        def __init__(self):
            self.ex_amplitude=0.0
            self.ex_length=0.0
        def set_ex_length(self,length):
            self.ex_length = length
            self.set_ex_pulse()
        def set_ex_amp(self, amp):
            self.ex_amplitude = amp
            self.set_ex_pulse()
        def set_ex_pulse(self):
            print (self.ex_amplitude, self.ex_length)
            sequence = [pg.p(None, readout_begin-self.ex_length), 
                        pg.p(rabi_channel, self.ex_length, pg.rect, self.ex_amplitude), 
                        pg.p(None, pause_length), 
                        pg.p('ro_trg', trg_length, pg.rect, 1), 
                        pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
            pg.set_seq(sequence)
            awg_tek.run()
    ps = pulse_setter()        
    measurement = sweep.sweep(adc_reducer, 
                              (lengths, ps.set_ex_length, 'Rabi pulse length', 's'), 
                              (ex_amps, ps.set_ex_amp, 'Rabi pulse amplitude'), 
                              filename=measurement_name)


#    measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)


#    annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
#                                                                                 fitted_parameters['freq'], 
#                                                                                 fitted_parameters['decay'])
#    save_pkl.save_pkl({'type':'Rabi','name': 'qubit' }, measurement_fitted, annotation=annotation, filename=measurement_name)

A Jupyter Widget

Started at:  Jul 06 2018 12:52:44
4
0.0 0.0


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:655: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


0.0 0.0
0.02 0.0
0.04 0.0
0.06 0.0
0.08 0.0
0.1 0.0
0.12 0.0
0.14 0.0
0.16 0.0
0.18 0.0
0.2 0.0
0.22 0.0
0.24 0.0
0.26 0.0
0.28 0.0
0.3 0.0
0.32 0.0
0.34 0.0
0.36 0.0
0.38 0.0
0.4 0.0
0.42 0.0
0.44 0.0
0.46 0.0
0.48 0.0
0.5 0.0
0.52 0.0
0.54 0.0
0.56 0.0
0.58 0.0
0.6 0.0
0.62 0.0
0.64 0.0
0.66 0.0
0.68 0.0
0.7 0.0
0.72 0.0
0.74 0.0
0.76 0.0
0.78 0.0
0.78 4e-09
0.0 4e-09
0.02 4e-09
0.04 4e-09
0.06 4e-09
0.08 4e-09
0.1 4e-09
0.12 4e-09
0.14 4e-09
0.16 4e-09
0.18 4e-09
0.2 4e-09
0.22 4e-09
0.24 4e-09
0.26 4e-09
0.28 4e-09
0.3 4e-09
0.32 4e-09
0.34 4e-09
0.36 4e-09
0.38 4e-09
0.4 4e-09
0.42 4e-09
0.44 4e-09
0.46 4e-09
0.48 4e-09
0.5 4e-09
0.52 4e-09
0.54 4e-09
0.56 4e-09
0.58 4e-09
0.6 4e-09
0.62 4e-09
0.64 4e-09
0.66 4e-09
0.68 4e-09
0.7 4e-09
0.72 4e-09
0.74 4e-09
0.76 4e-09
0.78 4e-09
0.78 8e-09
0.0 8e-09
0.02 8e-09
0.04 8e-09
0.06 8e-09
0.08 8e-09
0.1 8e-09
0.12 8e-09
0.14 8e-09
0.16 8e-09
0.18 8e-09
0.2 8e-09
0.22 8e-09
0.24 8e-09
0.26 8e-09
0.28 8e-09
0.3 8e-09
0.32 8e-09
0.34 8e-09


0.58 6.4e-08
0.6 6.4e-08
0.62 6.4e-08
0.64 6.4e-08
0.66 6.4e-08
0.68 6.4e-08
0.7 6.4e-08
0.72 6.4e-08
0.74 6.4e-08
0.76 6.4e-08
0.78 6.4e-08
0.78 6.8e-08
0.0 6.8e-08
0.02 6.8e-08
0.04 6.8e-08
0.06 6.8e-08
0.08 6.8e-08
0.1 6.8e-08
0.12 6.8e-08
0.14 6.8e-08
0.16 6.8e-08
0.18 6.8e-08
0.2 6.8e-08
0.22 6.8e-08
0.24 6.8e-08
0.26 6.8e-08
0.28 6.8e-08
0.3 6.8e-08
0.32 6.8e-08
0.34 6.8e-08
0.36 6.8e-08
0.38 6.8e-08
0.4 6.8e-08
0.42 6.8e-08
0.44 6.8e-08
0.46 6.8e-08
0.48 6.8e-08
0.5 6.8e-08
0.52 6.8e-08
0.54 6.8e-08
0.56 6.8e-08
0.58 6.8e-08
0.6 6.8e-08
0.62 6.8e-08
0.64 6.8e-08
0.66 6.8e-08
0.68 6.8e-08
0.7 6.8e-08
0.72 6.8e-08
0.74 6.8e-08
0.76 6.8e-08
0.78 6.8e-08
0.78 7.2e-08
0.0 7.2e-08
0.02 7.2e-08
0.04 7.2e-08
0.06 7.2e-08
0.08 7.2e-08
0.1 7.2e-08
0.12 7.2e-08
0.14 7.2e-08
0.16 7.2e-08
0.18 7.2e-08
0.2 7.2e-08
0.22 7.2e-08
0.24 7.2e-08
0.26 7.2e-08
0.28 7.2e-08
0.3 7.2e-08
0.32 7.2e-08
0.34 7.2e-08
0.36 7.2e-08
0.38 7.2e-08
0.4 7.2e-08
0.42 7.2e-08
0.44 7.2e-08
0.46 7.2e-08
0.48 7.2e-08
0

### Осцилляции Рамзея

In [22]:
target_freq_offset = -21e6
ex_amplitude = 0.4
ex_channels = ['iq_ex_00-1-01', 'iq_ex_00-1-10', 'iq_ex_00-2-11']
rabi_freqs = [48.79e6, 38.11e6, 15.72e6]
for ex_channel, rabi_freq in zip(ex_channels, rabi_freqs):
    measurement_name = 'Ramsey (target offset {0:4.2f} MHz), excitation channel {1}'.format(target_freq_offset/1e6, ex_channel)
    #pi2_pulse = 6e-9
    #0.25/15.5e6
    pi2_pulse = 0.25/rabi_freq

    delays = np.arange(0, 0.4e-6, 0.004e-6)

    readout_begin = np.max(delays)+pi2_pulse*2

    def set_delay(delay):
        awg_tek.set_nop(awg_tek.get_clock()/rep_rate)

        sequence = [pg.p(None, readout_begin-pi2_pulse),
                    pg.p(ex_channel, pi2_pulse, pg.rect, ex_amplitude), 
                    pg.p(None, delay), 
                    pg.p(ex_channel, pi2_pulse, pg.rect, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi)), 
                    pg.p('ro_trg', trg_length, pg.rect, 1), 
                    pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]

        pg.set_seq(sequence)
        awg_tek.run()
    measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay', 's'), filename=measurement_name)
    #measurement = sweep.sweep(adc_sz, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
    measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
    annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                                 fitted_parameters['freq'], 
                                                                                 fitted_parameters['decay'])
    save_pkl.save_pkl({'type':'Ramsey', 'name': 'qubit {}'.format(qubit_id)}, measurement_fitted, annotation=annotation, filename=measurement_name)

A Jupyter Widget

Started at:  Jul 06 2018 16:35:26
6


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2169: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2170: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2172: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)
D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:655: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 15 m 16.98 s
6


A Jupyter Widget

...


Elapsed time: 0 h 17 m 33.26 s
6


A Jupyter Widget

Started at:  Jul 06 2018 17:08:28
6


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2169: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2170: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2172: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)
D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:655: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 17 m 40.1 s
6


In [ ]:
save_pkl.save_pkl({'type':'Ramsey', 'name': 'qubit {}'.format(qubit_id)}, [0], annotation=None)

In [ ]:
save_pkl.save_pkl({'type':'Ramsey', 'name': 'qubit {}'.format(qubit_id)}, measurement_fitted, annotation=annotation)

### Затухание свободной индукции (T1)

In [25]:
ex_amplitude = 0.4
ex_channels = ['iq_ex_00-1-01', 'iq_ex_00-1-10', 'iq_ex_00-2-11']
rabi_freqs = [48.79e6, 38.11e6, 15.72e6]
for ex_channel, rabi_freq in zip(ex_channels, rabi_freqs):
    measurement_name = 'Decay, excitation channel {0}'.format(ex_channel)
    #pi2_pulse = 6e-9
    #0.25/15.5e6
    pi_pulse = 0.5/rabi_freq
    pause_length = 0
    delays = np.arange(0, 5e-6, 0.025e-6)
    readout_begin = np.max(delays)+pi_pulse
    def set_delay(delay):
        awg_tek.set_nop(awg_tek.get_clock()/rep_rate)

        sequence = [pg.p(None, readout_begin-ro_dac_length),
                    pg.p(ex_channel, pi_pulse, pg.rect, ex_amplitude), 
                    pg.p(None, delay), 
                    pg.p('ro_trg', trg_length, pg.rect, 1), 
                    pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]

        pg.set_seq(sequence)
        awg_tek.run()
    measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'delay', 's'), filename=measurement_name)

    measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_fit)
    #save_pkl.save_pkl({'type':'Decay'}, measurement_fitted)
    annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
    save_pkl.save_pkl({'type':'Decay', 'name': 'qubit {}'.format(qubit_id)}, measurement_fitted, annotation=annotation, filename=measurement_name)

A Jupyter Widget

Started at:  Jul 06 2018 17:31:15
6


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2169: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2170: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2172: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)
D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:655: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 30 m 56.55 s
[4.7822991710898291e-06, 0.003447906164823532, -0.006324044171275535, 0.015879640504248435, 0.016968829945710737, 0.003877104959148019, -0.007245481419046192, 0.012942607254690985, 0.018867771023713814]
6


A Jupyter Widget

...


Elapsed time: 0 h 35 m 22.21 s
[4.5584042519009751e-06, 0.003681177091847858, -0.005958351856255596, 0.017034988181224862, 0.017843709046398093, 0.0022730067377345394, -0.00652719256906, 0.013183887738831303, 0.017067207382273755]
6


A Jupyter Widget

Started at:  Jul 06 2018 18:37:47
6


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2169: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2170: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2172: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)
D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:655: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 35 m 33.43 s
[4.8247067935399438e-06, 0.0019017772201516802, -0.00546157593233791, 0.017050405424559732, 0.01606657497856974, 0.002705190766807041, -0.00667329085830832, 0.013294526329432507, 0.01740039133393051]
6


In [ ]:
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

### Спиновое эхо

In [ ]:
ex_amplitude = 0.2
pi2_pulse = 9e-9
pause_length = 15.98e-9
delays = np.linspace(0e-9, 500e-9, 101)
target_freq_offset = 10e6

readout_begin = np.max(delays)+pi2_pulse*2

def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-pi2_pulse),
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), 
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi)), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays*2, set_delay, 'Spin echo full delay'), filename='Spin echo')
#measurement = sweep.sweep(adc_sz, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Spin echo', 'name': 'qubit {} paramp {}'.format(qubit_id, 'on' if lo_amp.get_status() else 'off')}, measurement_fitted, annotation=annotation)

In [ ]:
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

In [ ]:
lo_amp.set_status(0)

In [ ]:
qubit_id

In [ ]:
adc.stop()

### Rabi amplitude oscillations

In [ ]:
ex_amps = np.linspace(0.75,1.0,21)
pause_length = 32e-9
pi2_length = 16e-9
sigma = 4e-9
num_pulses = 16
readout_begin = np.max((pi2_length+pause_length)*num_pulses)
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('iq_ex', pi2_length, pg.gauss_hd,  amp, 0, sigma,0)]*num_pulses+\
               [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    #print(sequence)
    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')

### gaussian $\pi/2$ pulse amplitudes

In [ ]:
#amp_x=0.554
#amp_y=0.554
#sigma = 2e-9
#amp_x=0.29
#amp_y=0.29
amp_x = 0.90
length = 16e-9
pause_length = 0e-9
sigma = 4e-9
amp_x_hd = 0.89
length_hd = 16e-9
sigma_hd = 4e-9
pause_length_hd = 8e-9
alpha_hd = -0.92e-9

### Ramsey with gauss pulses

In [ ]:
delays = np.linspace(0e-9, 2000e-9, 201)
target_freq_offset = 20e6

def set_delay(delay):
    sequence = [pg.p(None, readout_begin-length),
                pg.p('iq_ex', pi2_pulse, pg.gauss_hd, sigma, ex_amplitude, 0), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.gauss_hd, sigma, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi), 0), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

# Optimizing readout pulse settings

In [ ]:
#ex_calib_seq = [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma), pg.p(None, pause_length)]*2
ex_calib_seq = [pg.p('iq_ex', 18e-9, pg.rect, 0.2), pg.p(None, pause_length)]
ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
adc_sz = tomography_new.sz_measurer(adc, ex_calib_seq, ro_seq, pg)
adc_sz.save_last_samples = True
#adc_sz.dump_measured_samples = True
adc.set_software_nums_multi(1)
adc_sz.repeat_samples = 3

def target_min():
    F = 1-adc_sz.measure()['Calibrated ROC AUC']
    print ('fidelity: ', 1-F)
    return F

class ro_pulse_generator:
    def __init__(self, adc_sz, ro_amplitude, ro_dac_length):
        self.ro_amplitude = ro_amplitude
        self.ro_dac_length = ro_dac_length
        self.adc_sz = adc_sz
    def set_ro_freq(self, ro_freq):
        iq_ro.set_uncal_frequency(ro_freq)
        self.make_ro_seq()
    def set_paramp_pump_freq(self, pump_freq):
        lo_amp.set_frequency(pump_freq)
    def set_paramp_pump_power(self, pump_power):
        lo_amp.set_power(pump_power)
    def set_paramp_bias_current(self, paramp_bias):
        current_src2.set_current(paramp_bias)
    def make_ro_seq(self):
        adc_sz.ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', self.ro_dac_length, pg.rect, self.ro_amplitude)]
    def set_ro_length(self, ro_dac_length):
        self.ro_dac_length = ro_dac_length
        self.make_ro_seq()
        adc_sz.measurement_name = 'ro_dac_length {0}ns ro_dac_amiltude {1:4.2f}'.format(int(self.ro_dac_length*1e9), self.ro_amplitude)
    def set_ro_amplitude(self, ro_amplitude):
        self.ro_amplitude = ro_amplitude
        self.make_ro_seq()
        adc_sz.measurement_name = 'ro_dac_length {0}ns ro_dac_amiltude {1:4.2f}'.format(int(self.ro_dac_length*1e9), self.ro_amplitude)

ro_dac_lengths = np.linspace(0.05e-6,1.0e-6,20)
ro_dac_amplitudes = np.linspace(0.05,1.0,20)

ro_pulse_generator_inst = ro_pulse_generator(adc_sz, ro_amplitude, ro_dac_length)
#solution, score = sweep.optimize(target_min, (ro_pulse_generator_inst.set_ro_length, 3e-8), 
#                                             (ro_pulse_generator_inst.set_ro_amplitude, 0.9), maxfun=30)
#ro_dac_length = solution[0]
#ro_dac_amplitude = solution[1]
#

In [ ]:
sweep.sweep(adc_sz, (ro_dac_lengths, ro_pulse_generator_inst.set_ro_length, 'Readout pulse length'), 
                    (ro_dac_amplitudes, ro_pulse_generator_inst.set_ro_amplitude, 'Readout pulse amplitude'), filename='Readout fidelity with paramp')

## Optimizing readout pulse length, amplitude, frequency, paramteric amplifier pump and bias

In [ ]:
p0 = [700e-9, 0.5, iq_ro.get_frequency(), lo_amp.get_frequency(), lo_amp.get_power(), current_src2.get_current()]
dp = [50e-9, 0.05, 0.5e6, 3e6, 0.06, 3e-5]

initial_simplex = [p0]+[[p0[i]+(dp[i] if i == j else 0) for i in range(len(dp))] for j in range(len(dp))]

solution, score = sweep.optimize(target_min, (ro_pulse_generator_inst.set_ro_length, 700e-9), 
                                             (ro_pulse_generator_inst.set_ro_amplitude, 0.5),
                                             (ro_pulse_generator_inst.set_ro_freq, iq_ro.get_frequency()),
                                             (ro_pulse_generator_inst.set_paramp_pump_freq, lo_amp.get_frequency()),
                                             (ro_pulse_generator_inst.set_paramp_pump_power, lo_amp.get_power()),
                                             (ro_pulse_generator_inst.set_paramp_bias_current, current_src2.get_current()),
                                             initial_simplex=initial_simplex,
                                             maxfun=50)

### |0> & |1> readout calibration

In [ ]:
lo_amp.get_status()

In [ ]:
imp.reload(tomography_new)
imp.reload(awg_iq)
imp.reload(save_pkl)

In [ ]:
ro_dac_length = 600e-9
ro_dac_amplitude = 0.517
ro_delay_length = 10e-9
ro_delay_amplitude = 1.
#ex_calib_seq = [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]*2
ex_calib_seq = [pg.p('iq_ex', 59e-9, pg.rect, 0.5), pg.p(None, pause_length)]
#ex_calib_seq = [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma)]*2
#ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
ro_delay_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_delay_length, pg.rect, ro_delay_amplitude)]
adc_sz = tomography_new.sz_measurer(adc, ex_calib_seq, ro_seq, pg, ro_delay_seq = ro_delay_seq)
adc_sz.save_last_samples = True
adc_sz.repeat_samples = 1
#adc.set_software_nums_multi(2)
print(adc_sz.measure_delay('iq_ro'))
print(adc_sz.measure())
#sz_filter = adc_sz.calibrate()
#print('ROC AUC:', adc_sz.calib_roc_auc, 'Fidelity:', adc_sz.calib_fidelity)
#print('ROC AUC binary:', adc_sz.calib_roc_auc_binary, 'Fidelity binary:', adc_sz.calib_fidelity_binary) 

In [ ]:
adc_sz.dump_samples('ro_dac_length {0}ns ro_dac_amiltude {1:4.2f}'.format(int(ro_dac_length*1e9), ro_dac_amplitude))

In [ ]:
adc.stop()

In [ ]:
plt.figure()
plt.pcolormesh(np.real(adc_sz.samples[0,:,:]), vmin=15, vmax=25); plt.colorbar()
plt.figure()
plt.pcolormesh(np.real(adc_sz.samples[1,:,:]), vmin=15, vmax=25); plt.colorbar()

In [ ]:
plt.hist(adc_sz.calib_pred.T, bins=20)

In [ ]:
#plt.scatter(np.real(adc_sz.calib_usl_pred[0,:]), np.imag(adc_sz.calib_usl_pred[0,:]), color='r', s=1)
#plt.scatter(np.real(adc_sz.calib_usl_pred[1,:]), np.imag(adc_sz.calib_usl_pred[1,:]), color='b', s=1)
#plt.figure()
#plt.hist(adc_sz.calib_pred.T, bins=20)
plt.figure()
#plt.hist(np.real(adc_sz.calib_usl_pred.T), bins=20)
plt.plot(np.real(adc_sz.calib_bg-np.mean(adc_sz.calib_bg)))
plt.plot(np.imag(adc_sz.calib_bg-np.mean(adc_sz.calib_bg)))
plt.plot(np.real(adc_sz.calib_feature))
plt.plot(np.imag(adc_sz.calib_feature))

### Tomography

In [ ]:
observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}
proj_seq = {'Xo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq, 
                 'operator': observables['X']},
            'Yo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['Y']},
            '-Xo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-X']},
            '-Yo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-Y']},
            'Zo': {'pulses':ro_seq, 'operator':observables['Z']} }
reconstruction_basis={'x':{'operator':observables['X']},
                      'y':{'operator':observables['Y']},
                      'z':{'operator':observables['Z']}}
tomo = tomography_new.tomography(adc_sz, pg, proj_seq, reconstruction_basis=reconstruction_basis)

In [ ]:
proj_seq = {'Z': {'pulses':ro_seq, 'operator':observables['Z']}}
tomoz = tomography_new.tomography(adc_sz, pg, proj_seq)

In [ ]:
## tomography of |0> state
adc.set_software_nums_multi(20)
tomo.set_prepare_seq([])
tomo.measure()

In [ ]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]*2)
tomo.measure()

In [ ]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+
                     [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length),
                     pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, alpha_hd), pg.p(None, pause_length)]*10)
tomo.measure()

In [ ]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, -0.92e-9), pg.p(None, pause_length)]+
                     [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, -0.92e-9), pg.p(None, pause_length),
                     pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, -0.92e-9), pg.p(None, pause_length)]*10)
tomo.measure()

In [ ]:
## tomography of |0>+|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

In [ ]:
## tomography of |0>-|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, 1j*amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

In [ ]:
## tomography of -|0>-|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

In [ ]:
## tomography of -|0>+|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -1j*amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

# Randomized benchmarking

In [ ]:
import interleaved_benchmarking
imp.reload(interleaved_benchmarking)

bench = interleaved_benchmarking.interleaved_benchmarking(tomo)

unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}
bench.interleavers = {'X': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]])},
                      'Y': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 1j*amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]])},
                      '-X':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]])},
                      '-Y':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -1j*amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])},
                      'I':{'pulses':[], 'unitary':np.asarray([[1, 0], [0,1]])}}
bench.sequence_length = 10
bench.random_sequence_num = 6
bench.prepare_random_interleaving_sequences()
#bench.generate_interleaver_sequence_from_names()

In [ ]:
seq_lengths = np.arange(101)
sweep.sweep(bench, (seq_lengths, bench.set_sequence_length_and_regenerate, 'Gate number'))

In [ ]:
bench.target_gate = bench.interleavers['X']['pulses']
bench.target_gate_unitary = bench.interleavers['X']['unitary']
bench.target_gate_name = 'X (benchmarking)'
tfunc = lambda : np.mean(bench.measure()['Euclidean distance'])
class hd_pulse_generator:
    def __init__(self, pulse_setter, pg, channel):
        self.amp_x = -amp_x_hd
        self.amp_y = amp_x_hd
        self.sigma = sigma_hd
        self.length = length_hd
        self.pause_length = pause_length_hd
        self.alpha = alpha_hd
        
        self.pg = pg
        self.channel = channel
        self.pulse = self.pulse_gen()
        self.pulse_setter = pulse_setter
    def set_real_amplitude(self,x):
        self.amp_x = x
        self.pulse_setter(self.pulse_gen())
    def set_imag_amplitude(self,y):
        self.amp_y = y
        self.pulse_setter(self.pulse_gen())
    def set_sigma(self,s):
        self.sigma = s
        self.pulse_setter(self.pulse_gen())
    def set_length(self,l):
        self.length = l
        self.pulse_setter(self.pulse_gen())
    def set_pause_length(self, p):
        self.pause_length = p
        self.pulse_setter(self.pulse_gen())
    def set_alpha(self, a):
        self.alpha = a
        self.pulse_setter(self.pulse_gen())
    def pulse_gen(self):
        return [pg.p(self.channel, self.length, self.pg.gauss_hd, self.amp_x+1j*self.amp_y, 0, self.sigma, self.alpha), pg.p(None, self.pause_length)]
    
xg = hd_pulse_generator(bench.set_target_pulse, pg, 'iq_ex')

In [ ]:
sol, score = sweep.optimize(tfunc,  (xg.set_real_amplitude, xg.amp_x), 
                                    (xg.set_imag_amplitude, xg.amp_y),
                                    (xg.set_alpha, xg.alpha))

In [ ]:
amps = np.linspace(0.925, 0.975, 21)
alphas = np.linspace(-2e-9, 2e-9, 11)
sweep.sweep(bench, (amps, xg.set_real_amplitude, 'Amplitude'), (alphas, xg.set_alpha, 'HD DRAG coeff'), filename='Randomized interleaved benchmarging X-rotation')

In [ ]:
data = save_pkl.load_pkl('Euclidean distance Randomized interleaved benchmarging X-rotation', 'D:\\qtlab\\ReiData\\data\\2017-07-02\\23-03-34')
plt.figure()
plt.pcolormesh(data[1]['Euclidean distance'][1][1], data[1]['Euclidean distance'][1][0], data[1]['Euclidean distance'][2][:,:,5])
plt.colorbar()

In [ ]:
data[1]

In [ ]:
data[1]['Euclidean distance']

In [ ]:
meas = bench.measure()

In [ ]:
meas['Xo']

In [ ]:
import plotting
imp.reload(plotting)
plotting.plot_measurement(bench.reference_benchmark_result)

# Optimizing Drag

In [ ]:
num_interleved_pulses = 2
alphas = np.linspace(-1e-8, 1e-8, 201)
def set_drag (alpha):
    ramsey_half = [pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                 pg.p(None, pause_length_hd)]
    interleaved_identity = [pg.p('iq_ex', length_hd, pg.gauss_hd, -amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd),
                           pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd)]
    tomo.set_prepare_seq(ramsey_half+interleaved_identity*num_interleved_pulses+ramsey_half)
    
measurement = sweep.sweep(tomo, (alphas, set_drag, 'delta_freq'), filename='DRAG', output=False)

### Randomized Benchmarking

In [ ]:
pause_pulse = tomography.pause(pause_length, awg_channels)
pulses = {0:tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          1:tomography.ex_gauss_hd(amp_y_hd, -amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          2:tomography.ex_gauss_hd(-amp_x_hd, -amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          3:tomography.ex_gauss_hd(-amp_y_hd, amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi)}

unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}

observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}

initial = np.asarray([0, 1]).T
random_pulse_ids = np.arange(20)
num_gates = np.linspace(0, 100, 11)

sequences = []
pulse_sequences = []
theor_projs = {o:np.zeros((len(num_gates), len(random_pulse_ids))) for o in observables.keys()}
rho_t = np.zeros((len(num_gates), len(random_pulse_ids),2,2), dtype=np.complex)
psis = np.zeros((len(num_gates), len(random_pulse_ids),2), dtype=np.complex)
n=0

def set_num_gates(n_new):
    global n
    n = n_new

def set_random_sequence(id):
    sequence = np.random.randint (4, size = n).tolist()
    sequence_pulses = [j for i in [[pulses[i], pause_pulse] for i in sequence] for j in i]
    sequence_unitaries = [unitaries[i] for i in sequence]
    psi = initial.copy()
    
    for U in sequence_unitaries:
        psi = np.dot(U, psi)
    
    rho_t[n,id,:,:] = np.einsum('i,j->ij', np.conj(psi), psi)
    psis[n,id,:] = psi
    
    sequences.append(sequence)
    for o in observables.keys():
        theor_projs[o][n,id] = np.dot(np.dot(np.conj(psi), observables[o]), psi)
    
    tomo_hd.set_prepare_seq(sequence_pulses)
    pulse_sequences.append(sequence_pulses)
    
m = sweep.sweep(tomo_hd, (num_gates, set_num_gates, 'gate num'), (random_pulse_ids, set_random_sequence, 'random seq id'), filename='RB', output=False)
X = (m['X'][2]-m['-X'][2])/2.
Y = (m['Y'][2]-m['-Y'][2])/2.
Z = m['Z'][2]
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('nmi,nmji,nmj->nm', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

In [ ]:
plt.plot(m['Fidelity'][1]m['Fidelity'])

In [ ]:
rho_m

In [ ]:
plt.plot(m['Fidelity'][1][0], np.mean(m['Fidelity'][2],axis=1))
plt.xlabel('gate num')
plt.ylabel('fidelity')
plt.grid()

In [ ]:
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('mni,mnji,mnj->mn', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

In [ ]:
m

In [ ]:
fidelities

In [ ]:
psis

In [ ]:
rho_m

In [ ]:
rho_t

In [ ]:
measurement['theory'] = thoer_proj
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, measurement)

In [ ]:
print(theor_projs)
print(measurement)
print(psis)
print (sequences)
print (rho_t)
print (rho_m)

In [ ]:
np.dot(np.conj(psis[1]), np.dot(observables['Y'],psis[1]))

In [ ]:
num_pulses = np.round(np.arange(0, 41)).astype(np.int)

def set_num_pulses(n):
    tomoz.set_prepare_seq([tomography.ex_gauss(amp_x, length, sigma, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomoz, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss(amp_x_hd, length_hd, sigma_hd, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
ex_amplitude=0.25
lengths = np.linspace(0e-9, 200e-9, 51)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi x-axis pulse length'), filename='Rabi', output=False)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(measurement ['X'][2], measurement ['Y'][2], measurement ['Z'][2], c='r', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim([-0.6, 0.6])
ax.set_ylim([-0.6, 0.6])
ax.set_zlim([-0.6, 0.6])

In [ ]:
ex_amplitude = 0.25
pause_length = 16e-9
lengths = np.linspace(0e-9, 2000e-9, 2001)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(1j*ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi y-axis pulse length'), filename='Rabi', output=False)

### Optimizing phase between x and y pulses

In [ ]:
c_phase = 0
c_n = 0

adc.set_software_averages(1)
adc.set_software_nums_multi(3)

def set_n(n):
    global c_n
    global c_phase
    c_n = n 
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

def set_y_phase(phase):
    global c_n
    global c_phase
    c_phase = phase
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

phases = np.linspace(np.pi/2.*0.8, np.pi/2.*1.2, 11, endpoint=False)
num_pulses = np.arange(0,5)
measurement = sweep.sweep(tomo, (phases, set_y_phase, 'y-pulse phase'), (num_pulses, set_n, 'y pulse num'), filename='y-phase', output=False)

In [ ]:
phases[7]

In [ ]:
plt.plot(np.asarray(results).T)

In [ ]:
adc.stop()

In [ ]:
plt.close()

# load calibration data

In [ ]:
import pickle
f0 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-46\\Voltage ro calibration.pkl', 'rb')
f1 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-14\\Voltage ro calibration.pkl', 'rb')
cal0 = pickle.load(f0)
cal1 = pickle.load(f1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt4
cal0_nomean = cal0[1]['Voltage'][2][0,:,:]-np.mean(cal0[1]['Voltage'][2][0,:,:])
cal1_nomean = cal0[1]['Voltage'][2][1,:,:]-np.mean(cal0[1]['Voltage'][2][1,:,:])

cal0_mean = np.mean(cal0_nomean, axis=0)
cal1_mean = np.mean(cal1_nomean, axis=0)

In [ ]:
cal_mean = (cal0_mean+cal1_mean)/2.
cal_diff = (cal1_mean-cal0_mean)

plt.figure('Samples')
plt.plot(np.mean(cal0_nomean, axis=0), label='|0>')
plt.plot(np.mean(cal1_nomean, axis=0), label='|1>')
plt.legend()

plt.figure('Offset & feature')
plt.plot(cal_mean, label='mean')
plt.plot(cal_diff, label='diff')
plt.legend()

In [ ]:
plt.figure('Fourier domain')
plt.plot(np.abs(np.fft.fft(cal0_mean)), label='|0>')
plt.plot(np.abs(np.fft.fft(cal1_mean)), label='|1>') 
plt.plot(np.abs(np.fft.fft(cal_mean)), label='mean')
plt.plot(np.abs(np.fft.fft(cal_diff)), label='diff')
plt.legend()

feature = np.conj(cal_diff/np.sum(np.abs(cal_diff**2)))

In [ ]:
coeffs0 = np.dot(cal0_nomean-cal_mean, feature)
coeffs1 = np.dot(cal1_nomean-cal_mean, feature)
predictions = np.asarray([np.real(coeffs0), np.real(coeffs1)])

In [ ]:
hist_all, bins = np.histogram(predictions, bins='auto')
proba_points = (bins[1:]+bins[:-1])/2.
hists = []
for y in range(2):
    hists.append(np.histogram(predictions[y,:], bins=bins)[0])

hists = np.asarray(hists, dtype=float)
probabilities = hists/hist_all
naive_probabilities = np.asarray([proba_points<0, proba_points>0], dtype=float)
probabilities[np.isnan(probabilities)] = naive_probabilities[np.isnan(probabilities)]
predictor = lambda x: np.interp(x, proba_points, probabilities[1,:], left=0., right=1.)
calib_proba_points = proba_points
calib_proba = probabilities[1,:]
calib_hists = hists

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
roc = roc_curve([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))
roc_auc = roc_auc_score([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))

In [ ]:
plt.figure('ROC')
plt.plot(roc[0], roc[1])
plt.xlabel('True Positive Rate')
plt.ylabel('True Negative Rate')
roc_auc

In [ ]:
plt.figure('probability curve')
plt.plot(calib_proba_points, calib_proba)

In [ ]:
plt.figure('Readout hists')
plt.bar(proba_points-0.25, hists[0], width=0.05, label='|0>')
plt.bar(proba_points+0.25, hists[1], width=0.05, label='|1>')
plt.legend()

In [ ]:
plt.close()

In [ ]:
cal0_nomean.shape

In [ ]:
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal0_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal1_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')

In [ ]:
plt.plot(np.mean(cal0_nomean, axis=0))
plt.plot(np.mean(cal1_nomean, axis=0))

### extract samples from scan from their directories and put into a new directory

In [ ]:
import os
import glob
new_root = 'D:\\qtlab\\ReiData\\data\\2018-03-13\\Paramp-on-samples\\'
path = 'D:\\qtlab\\ReiData\\data\\2018-03-13\\Paramp-on-samples\\day2\\*\\*.pkl'
pickles = glob.glob(path)
for pickle in pickles:
    pickle_name = pickle[(pickle.rfind('\\')+1):]
    amplitude = 0.05
    pickle_renamed = pickle_name.replace('ro_dac_amiltude 1.00', 'ro_dac_amplitude {0:.2f}'.format(amplitude))
    i = 0
    while(os.path.exists(new_root+pickle_renamed)):
        amplitude += 0.05
        pickle_renamed = pickle_name.replace('ro_dac_amiltude 1.00', 'ro_dac_amplitude {0:.2f}'.format(amplitude))
        i += 1 
        if i> 100:
            raise(Exception('infinite loop '+pickle_renamed))
    #print (pickle_name)
    os.rename(pickle, new_root+pickle_renamed)

### zip samples

In [ ]:
import zipfile
sources = 'D:\\qtlab\\ReiData\\data\\2018-03-13\\Paramp-on-samples\\*.pkl'
pickles = glob.glob(sources)
for pickle in pickles:
    #pickle_name = pickle[(pickle.rfind('\\')+1):]
    print (pickle)
    zout = zipfile.ZipFile(pickle+'.zip', "w", zipfile.ZIP_DEFLATED)
    zout.write(pickle)
    zout.close()